In [3]:
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv, GCNConv

In [5]:
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

Processing...
Done!


In [6]:
class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super(GATNet, self).__init__()
        self.dropout = dropout
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GCNNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.6):
        super(GCNNet, self).__init__()
        self.dropout = dropout
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [7]:
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [8]:
def evaluate(model, data, mask):
    model.eval()
    out = model(data)
    pred = out[mask].max(1)[1]
    correct = pred.eq(data.y[mask]).sum().item()
    return correct / mask.sum().item()

In [9]:
if __name__ == '__main__':
    gat_hidden = 8
    gcn_hidden = 16
    heads = 8
    lr = 0.005
    weight_decay = 5e-4
    epochs = 200

    gat_model = GATNet(dataset.num_node_features, gat_hidden, dataset.num_classes, heads=heads).to(device)
    gcn_model = GCNNet(dataset.num_node_features, gcn_hidden, dataset.num_classes).to(device)

    gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=lr, weight_decay=weight_decay)
    gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = torch.nn.NLLLoss()

    best_val_acc = 0
    print('Training GAT model...')
    for epoch in range(1, epochs + 1):
        loss = train(gat_model, data, gat_optimizer, criterion)
        if epoch % 10 == 0:
            val_acc = evaluate(gat_model, data, data.val_mask)
            best_val_acc = max(best_val_acc, val_acc)
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')
    gat_test_acc = evaluate(gat_model, data, data.test_mask)
    print(f'GAT Test Accuracy: {gat_test_acc:.4f}\n')

    best_val_acc_gcn = 0
    print('Training GCN model...')
    for epoch in range(1, epochs + 1):
        loss = train(gcn_model, data, gcn_optimizer, criterion)
        if epoch % 10 == 0:
            val_acc = evaluate(gcn_model, data, data.val_mask)
            best_val_acc_gcn = max(best_val_acc_gcn, val_acc)
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')
    gcn_test_acc = evaluate(gcn_model, data, data.test_mask)
    print(f'GCN Test Accuracy: {gcn_test_acc:.4f}')


Training GAT model...
Epoch: 010, Loss: 1.2806, Val Acc: 0.7520
Epoch: 020, Loss: 0.9854, Val Acc: 0.7740
Epoch: 030, Loss: 0.7172, Val Acc: 0.7520
Epoch: 040, Loss: 0.5045, Val Acc: 0.7880
Epoch: 050, Loss: 0.5392, Val Acc: 0.7980
Epoch: 060, Loss: 0.4126, Val Acc: 0.7840
Epoch: 070, Loss: 0.5021, Val Acc: 0.7780
Epoch: 080, Loss: 0.4669, Val Acc: 0.7760
Epoch: 090, Loss: 0.5013, Val Acc: 0.7880
Epoch: 100, Loss: 0.4181, Val Acc: 0.7800
Epoch: 110, Loss: 0.3859, Val Acc: 0.7840
Epoch: 120, Loss: 0.4393, Val Acc: 0.7780
Epoch: 130, Loss: 0.3716, Val Acc: 0.7740
Epoch: 140, Loss: 0.3184, Val Acc: 0.7800
Epoch: 150, Loss: 0.4158, Val Acc: 0.7860
Epoch: 160, Loss: 0.4338, Val Acc: 0.7900
Epoch: 170, Loss: 0.3691, Val Acc: 0.7760
Epoch: 180, Loss: 0.3743, Val Acc: 0.7840
Epoch: 190, Loss: 0.4356, Val Acc: 0.7820
Epoch: 200, Loss: 0.3582, Val Acc: 0.7860
GAT Test Accuracy: 0.8030

Training GCN model...
Epoch: 010, Loss: 1.3961, Val Acc: 0.6160
Epoch: 020, Loss: 0.7765, Val Acc: 0.7300
Epoch

In [11]:
print('\nDiscussion:')
print(f'- GAT Test Accuracy: {gat_test_acc:.4f}')
print(f'- GCN Test Accuracy: {gcn_test_acc:.4f}')
print('- GAT uses attention heads to weight neighbor contributions, increasing parameters and potentially performance.')
print('- GCN has fewer parameters but may underperform when attention to specific neighbors is beneficial.')


Discussion:
- GAT Test Accuracy: 0.8030
- GCN Test Accuracy: 0.7860
- GAT uses attention heads to weight neighbor contributions, increasing parameters and potentially performance.
- GCN has fewer parameters but may underperform when attention to specific neighbors is beneficial.
